In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import gc
import numpy as np
import linkpred
from linkpred.evaluation import Pair


def find_common(g,k):
    """
    This function deletes all the nodes in g which are adiacent to less than k edges
    """
    nodes_to_remove = []
    start = g.number_of_nodes()
    # for each node delete all the nodes wich have less than 3 adiacent edges
    for node in g.nodes():
        if len(g.in_edges(node)) + len(g.out_edges(node))  < k:
            nodes_to_remove.append(node)
    for node in nodes_to_remove:
        g.remove_node(node)
    # print the number of nodes we deleted
    finish = g.number_of_nodes()
    print("{} nodes removed".format(start-finish))
    return g

def remove_useless_data(df):
    """
    Simple function used to remove from a dataframe all the rows in which the subreddit "parent"
    is equal to the subreddit "to"
    """
    df_1 = df[df["parent"] == df["to"]]
    # delete duplicate rows from dataframe
    new_df = df.drop(df_1.index)
    #delete useless data to free memory
    del df,df_1
    gc.collect()
    #reset the indexes in the new DataFrame
    new_df.reset_index(inplace=True,drop=True)
    return new_df


def remove_uncommon(g_1,g_2):
    """
    This function removes from a graph g_1 all the nodes that are not present in
    another graph g_2
    """
    nodes_to_remove = []
    for node in g_1.nodes():
        if node not in g_2.nodes():
            nodes_to_remove.append(node)
    for node in nodes_to_remove:
        g_1.remove_node(node)
    return g_1

def find_core(g_tr,g_te,k_train,k_test):
    """
    This function finds the core graph defined in the article ...

    """
    # find all the nodes adiacent at least to k edges
    g_tr = find_common(g_train,k_train)
    g_te = find_common(g_test,k_test)
    # remove all the nodes that are not present in both train and test graph
    g_tr_copy = g_tr.copy()
    g_te_copy = g_te.copy()
    g_tr1 = remove_uncommon(g_tr,g_te_copy)
    g_te1 = remove_uncommon(g_te,g_tr_copy)
    # merge the two graphs to obtain the core graph
    g_core = nx.compose(g_tr1,g_te1)
    return g_core

def accuracy_pred(g,results):
    """
    Simple function to compute the accuracy of a classifier
    """
    right_pred = 0

    for result in results:
        if (result[0],result[1]) in g.edges():
            right_pred = right_pred+1
        elif (result[1],result[0]) in g.edges():
            right_pred = right_pred+1
    accuracy = right_pred/(len(results))
    return accuracy

def predict_link(g,pred,predictions,name,**kwargs):
    """
    This function starts to predict return accuracy
    """
    pred_results = pred.predict(**kwargs)
    top_pred = pred_results.top(predictions)
    acc = accuracy_pred(g,top_pred)
    print("Accuracy {} predictor: {:.2f}%".format(name,acc*100))
    return acc


### Dividing the graph into train and test set

<p>The first thing I did was to split the dataset into two parts:<br>
1 Training set <br>
2 Test set <br>
    
I decided to use as split point the 12th of april 2021. 
This way the training-test split correspond roughly to the 80%-20% of the total data.</p>

In [2]:
#set the date that separates test and training set
START_DATE = 1618272000
# load and clean data
data = pd.read_csv("../scraping data/data/data_subreddit_cleaned.csv",index_col=0)
data = remove_useless_data(data)
#divide in training and test set
data_train = data[data["date"] < START_DATE]
data_test = data[data["date"] >= START_DATE]
print("Number of unique posts {}".format(len(pd.unique(data["title"]))))
print("Edges in trainig set {}\nEdges in test set {}".format(data_train.shape[0],data_test.shape[0]))

Number of unique posts 120747
Edges in trainig set 159926
Edges in test set 39279


### Finding the core

<p>Dividing the dataset into two parts is not enought.<br> 
What i did was to delete all the nodes from the training and test set that are not adiacent to at least 3 nodes. <br>
This way I eliminated all the subreddit that are not likely to interact with each other.<br>
Lastly i created a new graph that contains nodes that are present both in the training and test set.<br>
This graph is called core and rapresents the most active subreddits.</p>

In [4]:
# convert into nx Graphs
g_train = nx.convert_matrix.from_pandas_edgelist(data_train,source = "parent",target="to",
                                                edge_attr=True, create_using=nx.MultiDiGraph())

g_test = nx.convert_matrix.from_pandas_edgelist(data_test,source = "parent",target="to",
                                                 edge_attr=True, create_using=nx.MultiDiGraph())
#find core
core= find_core(g_train,g_test,3,3)
e_new = g_test.number_of_edges()
e_old = g_train.number_of_edges()
print("Nodes in core {}".format(core.number_of_nodes()))
print("Edges in core {}".format(core.number_of_edges()))
print("Edges in E_old {}\nEdges in E_new {}".format(e_old,e_new))

12721 nodes removed
6232 nodes removed
Nodes in core 3364
Edges in core 107167
Edges in E_old 93847
Edges in E_new 27044


In [5]:
accuracy = []
classifier = []
dictionary = {}

### Link Prediction

<p>After finding the core graph i started the prediction phase.<br>
Each predictor returns a score between two nodes u,v that rapresents how likely an edge (u,v) may form in the future.<br>
I selected the edges that return the highest score and then verified the accuracy of the prediction.</p>

In [6]:
rnd = linkpred.predictors.Random(g_train, excluded=g_train.edges())
label = "Random"
accuracy.append(predict_link(g_test,rnd,e_new,label))
classifier.append(label)

Accuracy Random predictor: 0.10%


In [7]:
cn = linkpred.predictors.CommonNeighbours(g_train, excluded=g_train.edges())
label = "Common Neighbours"
accuracy.append(predict_link(g_test,cn,e_new,label))
classifier.append(label)

Accuracy Common Neighbours predictor: 3.01%


In [8]:
jc = linkpred.predictors.Jaccard(g_train, excluded=g_train.edges())
label = "Jaccard"
accuracy.append(predict_link(g_test,jc,e_new,label))
classifier.append(label)

Accuracy Jaccard predictor: 2.54%


In [10]:
betas = [0.05,0.005,0.0005]
for beta in betas:
    kz = linkpred.predictors.Katz(g_train, excluded=g_train.edges())
    label = "Katz, beta: {}".format(beta)
    accuracy.append(predict_link(g_test,kz,e_new,label,beta=beta))
    classifier.append(label)

Computing matrix powers: [############################################################] 5/5
Accuracy Katz, beta: 0.05 predictor: 3.26%
Computing matrix powers: [############################################################] 5/5
Accuracy Katz, beta: 0.005 predictor: 4.05%
Computing matrix powers: [############################################################] 5/5
Accuracy Katz, beta: 0.0005 predictor: 4.30%


In [11]:
gd = linkpred.predictors.GraphDistance(g_train, excluded=g_train.edges())
label = "Graph Distance"
accuracy.append(predict_link(g_test,gd,e_new,label,alpha=0,weight=None))
classifier.append(label)

Accuracy Graph Distance predictor: 1.12%


In [12]:
alphas = [0.01,0.05,0.15,0.30,0.50]
for alpha in alphas:
    rpg = linkpred.predictors.RootedPageRank(g_train, excluded=g_train.edges())
    label = "Rooted page rank, alpha: {}".format(alpha)
    accuracy.append(predict_link(g_test,rpg,e_new,label,alpha=alpha,weight=None))
    classifier.append(label)

[############################################################] 3364/3364
Accuracy Rooted page rank, alpha: 0.01 predictor: 2.78%
[############################################################] 3364/3364
Accuracy Rooted page rank, alpha: 0.05 predictor: 2.80%
[############################################################] 3364/3364
Accuracy Rooted page rank, alpha: 0.15 predictor: 2.80%
[############################################################] 3364/3364
Accuracy Rooted page rank, alpha: 0.3 predictor: 2.75%
[############################################################] 3364/3364
Accuracy Rooted page rank, alpha: 0.5 predictor: 2.75%


In [13]:
simrank = linkpred.predictors.SimRank(g_train, excluded=g_train.edges())
simrank_results = simrank.predict(weight=None)
label = "Simrank"
accuracy.append(predict_link(g_test,simrank,e_new,label,weight=None))
classifier.append(label)

Accuracy Simrank predictor: 0.26%


In [14]:
dictionary["Classifier"] = classifier
dictionary["Accuracy"] = accuracy
clf_results = pd.DataFrame(dictionary)
clf_results.to_csv("clf_results.csv")
print (clf_results)

                       Classifier  Accuracy
0                          Random  0.000998
1               Common Neighbours  0.030099
2                         Jaccard  0.025366
3                Katz, beta: 0.05  0.032577
4               Katz, beta: 0.005  0.040527
5              Katz, beta: 0.0005  0.042967
6                  Graph Distance  0.011167
7   Rooted page rank, alpha: 0.01  0.027770
8   Rooted page rank, alpha: 0.05  0.027954
9   Rooted page rank, alpha: 0.15  0.027954
10   Rooted page rank, alpha: 0.3  0.027511
11   Rooted page rank, alpha: 0.5  0.027548
12                        Simrank  0.002625


### Observations

<p>Even tough we achieved on avarage a performane 10 times better than a random predictor, overall the results are not good. 
At most we achieved an accuracy of 4.30%.
This is due to different reasons:<br>
1. The internet is imprevedible: <br>
New trends,memes and topic of discussions may arise at any moment without notice.<br>
2. We used posts in the training set that are too old:<br>
The majority of the crossposts are fairily new, infact the test set is composed of posts at most two weeks old while the  training set contains posts from 2017. Since the internet changes so quickly, relationship between subreddits that are this old, may lead to wrong results.<br>

To test this hypotesis I repeated the precedent steps considering only the posts posted from march 2021 onwards</p>

In [28]:
    #set the date that separates test and training set
    START_DATE = 1618631200
    # load and clean data
    data = pd.read_csv("../scraping data/data/data_subreddit_cleaned.csv",index_col=0)
    data = remove_useless_data(data)
    data = data[data["date"] > 1614556800]
    #divide in training and test set
    data_train = data[data["date"] < START_DATE]
    data_test = data[data["date"] >= START_DATE]
    print("Number of unique posts {}".format(len(pd.unique(data["title"]))))
    print("Edges in trainig set {}\nEdges in test set {}".format(
                    data_train.shape[0],data_test.shape[0]))

    # convert into nx Graphs
    g_train = nx.convert_matrix.from_pandas_edgelist(data_train,source = "parent",target="to",
                                                     edge_attr=True, create_using=nx.MultiDiGraph())

    g_test = nx.convert_matrix.from_pandas_edgelist(data_test,source = "parent",target="to",

                                                     edge_attr=True, create_using=nx.MultiDiGraph())
    #find core
    core= find_core(g_train,g_test,3,3)
    e_new = g_test.number_of_edges()
    e_old = g_train.number_of_edges()
    print("Nodes in core {}".format(core.number_of_nodes()))
    print("Edges in core {}".format(core.number_of_edges()))
    print("Edges in E_old {}\nEdges in E_new {}".format(e_old,e_new))

Number of unique posts 62387
Edges in trainig set 68947
Edges in test set 28366
7853 nodes removed
5353 nodes removed
Nodes in core 2574
Edges in core 50142
Edges in E_old 40528
Edges in E_new 17946


In [29]:
cn = linkpred.predictors.CommonNeighbours(g_train, excluded=g_train.edges())
label = "Common Neighbours"
accuracy.append(predict_link(g_test,cn,e_new,label))
classifier.append(label)

Accuracy Common Neighbours predictor: 4.25%


In [30]:
jc = linkpred.predictors.Jaccard(g_train, excluded=g_train.edges())
label = "Jaccard"
accuracy.append(predict_link(g_test,jc,e_new,label))
classifier.append(label)

Accuracy Jaccard predictor: 3.62%


In [31]:
betas = [0.05,0.005,0.0005]
for beta in betas:
    kz = linkpred.predictors.Katz(g_train, excluded=g_train.edges())
    label = "Katz, beta: {}".format(beta)
    accuracy.append(predict_link(g_test,kz,e_new,label,beta=beta))
    classifier.append(label)

Computing matrix powers: [############################################################] 5/5
Accuracy Katz, beta: 0.05 predictor: 4.23%
Computing matrix powers: [############################################################] 5/5
Accuracy Katz, beta: 0.005 predictor: 5.09%
Computing matrix powers: [############################################################] 5/5
Accuracy Katz, beta: 0.0005 predictor: 5.08%


<p>This way I obtained better results but the overall accuracy is still too low.</p>